In [28]:
import MeCab
import re
import pandas as pd
import numpy as np

mecab = MeCab.Tagger()

In [29]:
def preprocessing(document): #특수문자 기본 전처리 함수(토큰화할때 마침표제거, 숫자변환)
    docu = []
    for doc in document:
        doc = re.sub('[^가-힣0-9\s.]','',doc)
        doc = re.sub('[-=+,.#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',doc)
        doc = re.sub('[0-9]+','num',doc) #숫자를 special token num으로 치환
        pattern = re.compile(r'\s+') #중복띄어쓰기 제거
        doc = re.sub(pattern,' ',doc)
        doc = doc.replace('\xa0','') 
        doc = doc.replace('\n','')
        docu.append(doc)

    return docu

In [30]:
df = pd.read_csv('./data/crawling/preprocessed/df_new.csv',index_col=0)
df_tokenized = pd.read_csv('./data/preprocessing/preprocessed/df_tokenized.csv', index_col=0)
df.str = preprocessing(df.str) #특수문자를 이전에 제거해줬지만 숫자를 num으로 처리하는과정 포함

한국어 형태소 분석기인 mecab을 가지고 Tokenization을 해주었습니다. 명사만 따로 추출하여 wordcloud로 확인하기 위해서 명사를 따로 추출했고 토큰화가 완료된 문서들을 str_token의 형태로 추가했습니다. 유사도 분석에 있어서 모든 형태소를 사용하느냐 조사나 어미를 제외하느냐에 따라서 차이가 있을것 같아서 ko-mecab이 제공하는 pos tags를 참고하여 조사나 어미를 제외하여 따로 토큰화를 하였습니다.

'EP','EF','EC','JKS','JKC','JKG','JKO','JKB','JC','JX'이 이에 해당합니다.

In [33]:
def mecab_types(text):   # konlpy의 mecab output 형태와 같게 만들어주는 함수
    morphs = []
    
    pattern = re.compile(".*\t[A-Z]+") 
    
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    for token in temp:
        morphs.append(token[1])
    
    return morphs

In [34]:
def mecab_morphs(text):   # 형태소 단위로 tokenization
    
    morphs = []
    
    pattern = re.compile(".*\t[A-Z]+") 
    
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    for token in temp:
        morphs.append(token[0])
    
    return morphs

In [35]:
def mecab_except_josa(text): #조사와 어미 tag 제외하고 반환
    
    morphs = []
    pattern = re.compile(".*\t[A-Z]+") 
    
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    
    josa_ls = ['EP','EF','EC','JKS','JKC','JKG','JKO','JKB','JC','JX']
        
    for token in temp:
        if token[1] in josa_ls:
            pass
        else:
            morphs.append(token[0])
    
    return morphs

In [36]:
def mecab_nouns(text):#명사만 반환
    morphs = []
    pattern = re.compile(".*\t[A-Z]+")
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    for token in temp:
        if token[1] == 'NNG':
            morphs.append(token[0])
    return morphs

In [37]:
df.head()

,연도,crp_nm,crp_cd,rpt_nm,rcp_no,rcp_dt,rmk,str
0,2019,삼양홀딩스,70,사업보고서 (2018.12),20190401004413,20190401,정연,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항당사가 본 경영진단의견서...
1,2018,삼양홀딩스,70,사업보고서 (2017.12),20180402004217,20180402,연,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항당사가 본 경영진단의견서...
2,2017,삼양홀딩스,70,사업보고서 (2016.12),20170331003683,20170331,연,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 당사가 본 경영진단의견...
3,2016,삼양홀딩스,70,사업보고서 (2015.12),20160330002684,20160330,정연,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 당사가 본 경영진단의견...
4,2015,삼양홀딩스,70,사업보고서 (2014.12),20150331002399,20150331,연,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 당사가 본 경영진단의견...


In [38]:
#형태소 기준 토큰화
kk = [mecab_morphs(x) for x in df['str']]
kl = [] #리스트안에 형태소분석된 토큰들이 들어있는 형태라서 str로 바꿔서 새 컬럼에 추가해준다
for k in kk:
    kl.append(' '.join(k))
df['str_tokens'] = kl
df = df.loc[:,('연도','crp_nm','rpt_nm','rcp_no','rcp_dt','str','str_tokens')]

#명사만
nouns_ls = [mecab_nouns(x) for x in df['str']]
ke=[]
for k in nouns_ls:
    ke.append(' '.join(k))
df['str_nouns'] = ke

#조사,어미 제외
ex_josa = [mecab_except_josa(x) for x in df['str']]
kj = []
for k in ex_josa:
    kj.append(' '.join(k))
df['str_ex_josa'] = kj

In [39]:
df

,연도,crp_nm,rpt_nm,rcp_no,rcp_dt,str,str_tokens,str_nouns,str_ex_josa
0,2019,삼양홀딩스,사업보고서 (2018.12),20190401004413,20190401,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항당사가 본 경영진단의견서...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...
1,2018,삼양홀딩스,사업보고서 (2017.12),20180402004217,20180402,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항당사가 본 경영진단의견서...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...
2,2017,삼양홀딩스,사업보고서 (2016.12),20170331003683,20170331,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 당사가 본 경영진단의견...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...
3,2016,삼양홀딩스,사업보고서 (2015.12),20160330002684,20160330,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 당사가 본 경영진단의견...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...
4,2015,삼양홀딩스,사업보고서 (2014.12),20150331002399,20150331,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 당사가 본 경영진단의견...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 본 경영 진단 ...
...,...,...,...,...,...,...,...,...,...
1125,2018,LS산전,사업보고서 (2017.12),20180330003041,20180330,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항 연결회사가 동 사업보고...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 연결 회사...,이사 경영 진단 분석 의견 예측 정보 주 사항 연결 회사 사업 보고서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 연결 회사 동 사업 ...
1126,2019,삼성증권,사업보고서 (2018.12),20190401005163,20190401,이사의 경영진단 및 분석의견num 예측정보에 대한 주의사항당사가 동 사업보고서에서...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 대한 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 사업 보고서 미래 발생 예상 예...,이사 경영 진단 및 분석 의견 num 예측 정보 대한 주 사항 당사 동 사업 보고서...
1127,2017,삼성카드,사업보고서 (2016.12),20170331005195,20170331,이사의 경영진단 및 분석의견num 예측정보에 따른 주의사항 당사가 동 경영진단 의...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 따른 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 따른 주 사항 당사 동 경영 진단 ...
1128,2018,삼성카드,사업보고서 (2017.12),20180402003595,20180402,이사의 경영진단 및 분석의견num 예측정보에 따른 주의사항 당사가 동 경영진단 의...,이사 의 경영 진단 및 분석 의견 num 예측 정보 에 따른 주 의 사항 당사 가 ...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 경영 진단 의견서 미래 발생 예...,이사 경영 진단 및 분석 의견 num 예측 정보 따른 주 사항 당사 동 경영 진단 ...


In [40]:
df.to_csv('./data/preprocessing/preprocessed/new_tokenized.csv')